<a href="https://colab.research.google.com/github/Svensone/Web-Scraping/blob/master/WebScraping_Covid_Bali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary

1) get covid cases per regency in Bali

2) additional data for Bali Province
- population and area of regencies
- hospital beds

3) comparison on national level


In [2]:
## install necessary dependencies etc.

!pip install scrapy

     |████████████████████████████████| 245kB 5.1MB/s 
     |████████████████████████████████| 3.2MB 6.9MB/s 
     |████████████████████████████████| 61kB 5.7MB/s 
     |████████████████████████████████| 2.6MB 28.7MB/s 
     |████████████████████████████████| 245kB 40.2MB/s 
     |████████████████████████████████| 3.1MB 36.5MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7765 sha256=f3e8c2343869092bfcd044b424611b2195e9640f5b6c871c973c4eb776536732
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11517 sha256=796204185a1b6d460d940aa4126e81eafa1e0fa2b08cda1dc41d923cb3cc3cb7
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
Successfully built protego PyD

In [11]:
## import dependencies

# Web Scraping
# import scrapy
import requests #better than in-built urlib
import html5lib
from bs4 import BeautifulSoup
import time
import csv

# Data processing
import pandas as pd


## 1. Covid Cases for Bali on Regencies level

### scrapying data

In [4]:
url = "https://infocorona.baliprov.go.id/API/pendataan/laporan-harian-01.php"
headers = {'User-Agent': 'Mozilla/5.0'}

In [5]:
response = requests.get(url)

In [6]:
response.status_code

200

In [7]:
response.content

b'<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" >\n<link rel="stylesheet" href="https://use.fontawesome.com/releases/v5.7.2/css/all.css">\n<link rel="stylesheet" href="https://pendataan.baliprov.go.id/assets/css/style.css?v=15">\n<link rel="stylesheet" href="https://pendataan.baliprov.go.id/assets/css/widget-custom.css?v=1">\n\n\n<section class="section">\n<div class="section-header" style="margin-top: 18px;"><h1>Laporan Harian Berdasarkan Sumber Penularan pada Tiap Wilayah</h1></div>\n<div class="section-body"><div class="card"><div class="card-body">\n\n\n<div class=\'alert alert-info\' role=\'alert\'>\n    <b>Informasi!</b> Menampilkan data laporan tanggal 20 Nov 2020 \n</div>\n<div class="table-responsive text-center">\n    <table class="table table-bordered table-striped text-center" border="1">\n    <thead>\n        <tr>\n            <th rowspan="3">NO</th>\n            <th rowspan="3">KABUPATEN / KOTA</th>\n            <th

In [12]:
soup = BeautifulSoup(response.content, 'html5lib' ) # or 'html.parser'
soup

<html><head><link href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" rel="stylesheet"/>
<link href="https://use.fontawesome.com/releases/v5.7.2/css/all.css" rel="stylesheet"/>
<link href="https://pendataan.baliprov.go.id/assets/css/style.css?v=15" rel="stylesheet"/>
<link href="https://pendataan.baliprov.go.id/assets/css/widget-custom.css?v=1" rel="stylesheet"/>


</head><body><section class="section">
<div class="section-header" style="margin-top: 18px;"><h1>Laporan Harian Berdasarkan Sumber Penularan pada Tiap Wilayah</h1></div>
<div class="section-body"><div class="card"><div class="card-body">


<div class="alert alert-info" role="alert">
    <b>Informasi!</b> Menampilkan data laporan tanggal 20 Nov 2020 
</div>
<div class="table-responsive text-center">
    <table border="1" class="table table-bordered table-striped text-center">
    <thead>
        <tr>
            <th rowspan="3">NO</th>
            <th rowspan="3">KABUPATEN / KOTA</th>
            <

## extract relevant data

In [13]:
covid_tables = soup.find_all('table')

In [14]:
len(covid_tables)

1

In [15]:
## get update-date

## MISSING 
## write code to check if updated

## if not yet updated div shows:
## \nInformasi! Menampilkan data laporan tanggal 19 Nov 2020 . Karena data tanggal 20 Nov 2020 belum dipublish.\n

## if updated div shows:
## 

dateUpdated = soup.find('div', class_='alert alert-info').get_text()

# print(str(dateUpdated[-11:-2]))
if dateUpdated[-11:-2]=='dipublish':
  print('No new update yet available')
else:
  date = dateUpdated[-13:-2]
date

'20 Nov 2020'

In [16]:
## extract rows of table

table = covid_tables[0]
rows = table.find_all('tr')
rows

[<tr>
             <th rowspan="3">NO</th>
             <th rowspan="3">KABUPATEN / KOTA</th>
             <th colspan="10">SUMBER PENULARAN POSITIF COVID-19 DI BALI</th>
         </tr>, <tr>
             <th colspan="2">PPLN / PMI</th>
             <th colspan="2">PPDN</th> 
             <th colspan="2">Transmisi Lokal</th>
             <th colspan="2">Lainnya</th>
             <th colspan="2">TOTAL</th>
         </tr>, <tr>
             <th>+</th>
             <th>JMLH</th>
             <th>+</th>
             <th>JMLH</th>
             <th>+</th>
             <th>JMLH</th>
             <th>+</th>
             <th>JMLH</th>
             <th>+</th>
             <th>JMLH</th>
         </tr>, <tr><td>1</td><td style="text-align: left;">JEMBRANA</td><td>0</td><td>26</td><td>0</td><td>11</td><td>1</td><td>460</td><td>0</td><td>0</td><td>1</td><td>497</td></tr>, <tr><td>2</td><td style="text-align: left;">TABANAN</td><td>0</td><td>21</td><td>0</td><td>8</td><td>8</td><td>939</td><td>0</td>

In [17]:
## write data to csv file

## save file in folder and add day-date (search how to )
path = '/content/drive/MyDrive/Colab Notebooks/EDA and other Data Analysis w. Visualization/'
csvFile = open(path +'CovidBali.csv', 'a')
writer = csv.writer(csvFile)

try:
  # write update-date
  writer.writerow(['Data from', date, 'from https://infocorona.baliprov.go.id/'])

  # get cell data
  for row in rows:
    csvRow = []
    for cell in row.find_all(['td', 'th']):
      csvRow.append(cell.get_text())
    writer.writerow(csvRow)
finally:
  csvFile.close()

In [19]:
## check csv File

with open(path +'CovidBali.csv', 'r') as f:
  for row in f:
    print(row)

﻿Data from;19 Nov 2020;from https://infocorona.baliprov.go.id/;;;;;;;;;

# NO;KABUPATEN / KOTA;SUMBER PENULARAN POSITIF COVID-19 DI BALI;;;;;;;;;

# PPLN / PMI;PPDN;Transmisi Lokal;Lainnya;TOTAL;;;;;;;

# +;JMLH;+;JMLH;+;JMLH;+;JMLH;+;JMLH;;

# 1;JEMBRANA;0;26;0;11;5;459;0;0;5;496

# 2;TABANAN;0;21;0;8;15;931;0;0;15;960

# 3;BADUNG;0;27;0;4;15;2229;0;0;15;2260

# 4;DENPASAR;0;48;0;21;28;3527;0;0;28;3596

# 5;GIANYAR;0;29;0;12;14;1685;0;0;14;1726

# 6;BANGLI;0;58;0;3;0;808;0;0;0;869

# 7;KLUNGKUNG;0;19;0;0;0;891;0;0;0;910

# 8;KARANGASEM;0;23;0;0;6;944;0;0;6;967

# 9;BULELENG;0;41;0;6;5;1097;0;0;5;1144

# 10;KABUPATEN LAINNYA;0;5;0;30;0;11;0;0;0;46

# 11;WARGA NEGARA ASING;0;8;0;2;0;21;0;0;0;31

# TOTAL;0;305;0;97;88;12603;0;0;88;13005;

Data from,20 Nov 2020,from https://infocorona.baliprov.go.id/

NO,KABUPATEN / KOTA,SUMBER PENULARAN POSITIF COVID-19 DI BALI

PPLN / PMI,PPDN,Transmisi Lokal,Lainnya,TOTAL

+,JMLH,+,JMLH,+,JMLH,+,JMLH,+,JMLH

1,JEMBRANA,0,26,0,11,1,460,0,0,1,497

2,TABA

In [ ]:
## 2. additional Data

### Population and Area


In [23]:
url = 'https://id.wikipedia.org/wiki/Daftar_kabupaten_dan_kota_di_Bali#cite_note-:0bali-1'
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}

response = requests.get(url)
print(response.status_code)

soup = BeautifulSoup(response.content, "html5lib")

#find table and extract data
table2 = soup.find_all('table', class_="wikitable sortable mw-collapsible")
table2

200


[<table class="wikitable sortable mw-collapsible">
 <tbody><tr>
 <th>No.
 </th>
 <th>Kabupaten/kota
 </th>
 <th>Pusat pemerintahan
 </th>
 <th>Bupati/wali kota
 </th>
 <th>Luas wilayah (km<sup>2</sup>)<sup class="reference" id="cite_ref-:0bali_1-0"><a href="#cite_note-:0bali-1">[1]</a></sup>
 </th>
 <th>Jumlah penduduk (2017)<sup class="reference" id="cite_ref-:0bali_1-1"><a href="#cite_note-:0bali-1">[1]</a></sup>
 </th>
 <th>Kecamatan
 </th>
 <th>Kelurahan/desa
 </th>
 <th>Logo <div class="center"><div class="floatnone"><a class="image" href="/wiki/Berkas:Coat_of_arms_of_Bali.svg"><img alt="" data-file-height="482" data-file-width="506" decoding="async" height="57" src="//upload.wikimedia.org/wikipedia/commons/thumb/1/10/Coat_of_arms_of_Bali.svg/60px-Coat_of_arms_of_Bali.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/1/10/Coat_of_arms_of_Bali.svg/90px-Coat_of_arms_of_Bali.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/1/10/Coat_of_arms_of_Bali.svg/120px

In [33]:

## create df
## get data from the 'th' and 'td' tags

data = []
for div in table2:
  rows = div.find_all("tr")
  for row in rows:
    data.append(row.text)
    
data


['\nNo.\n\nKabupaten/kota\n\nPusat pemerintahan\n\nBupati/wali kota\n\nLuas wilayah (km2)[1]\n\nJumlah penduduk (2017)[1]\n\nKecamatan\n\nKelurahan/desa\n\nLogo \n\nPeta lokasi\n',
 '\n1\nKabupaten Badung\nMangupura\nI Nyoman Giri Prasta\n\n418,62\n\n468.346\n6\n16/46\n\n\n\n',
 '\n2\nKabupaten Bangli\nBangli\nI Made Gianyar\n\n490,71\n\n264.945\n4\n4/68\n\n\n\n',
 '\n3\nKabupaten Buleleng\nSingaraja\nPutu Agus Suradnyana\n\n1.364,73\n\n814.356\n9\n19/129\n\n\n\n',
 '\n4\nKabupaten Gianyar\nGianyar\nI Made Agus Mahayastra\n\n368,00\n\n492.757\n7\n6/64\n\n\n\n',
 '\n5\nKabupaten Jembrana\nNegara\nI Putu Artha\n\n841,80\n\n323.211\n5\n10/41\n\n\n\n',
 '\n6\nKabupaten Karangasem\nAmlapura\nI Gusti Ayu Mas Sumatri\n\n839,54\n\n545.389\n8\n3/75\n\n\n\n',
 '\n7\nKabupaten Klungkung\nSemarapura\nI Nyoman Suwirta\n\n315,00\n\n215.852\n4\n6/53\n\n\n\n',
 '\n8\nKabupaten Tabanan\nTabanan\nNi Putu Eka Wiryastuti\n\n1.013,88\n\n466.647\n10\n-/133\n\n\n\n',
 '\n9\nKota Denpasar\n-\nIda Bagus Rai Dh

In [32]:
data_clean = []
for item in data:
  for td in item:
    data = item.find_all('td').get_text()
    data = data.rstrip()
    data = data.replace('/', ' ')
  ## clean data from '[1]' Source link
  # regex1 = /\[.\]/g
  # data = data.replace(regex1, '')
    data_clean.append(data)

data_clean

AttributeError: ignored

In [21]:

## create df
## get 'th' data for columns name
df = pd.DataFrame(columns=[header])
df

,No.,Kabupaten kota,Pusat pemerintahan,Bupati wali kota,Luas wilayah (km2)[1],Jumlah penduduk (2017)[1],Kecamatan,Kelurahan desa,Logo,Peta lokasi


In [22]:
## save for later
columnNames = header
columnNames

['No.',
 'Kabupaten kota',
 'Pusat pemerintahan',
 'Bupati wali kota',
 'Luas wilayah (km2)[1]',
 'Jumlah penduduk (2017)[1]',
 'Kecamatan',
 'Kelurahan desa',
 'Logo',
 'Peta lokasi']